In [43]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
#@title
!wget -O uber.csv http://51.91.120.89/uber.csv

In [45]:
df = pd.read_csv("uber.csv", sep=';', header=None, na_values=['None'], usecols=[0,1,2,3])
df.columns = ['ts','name','price','multiplier']
df.ts = pd.to_datetime(df.ts, format='%Y%m%d%H%M')
df.dropna(inplace=True)
df.set_index('ts', inplace=True)
df.sort_values('ts', inplace=True)
df.sample(n=10)

,name,price,multiplier
ts,,,
2020-08-08 20:00:00,UberX Protection,23.01,1.6
2020-08-16 09:44:00,Van,27.66,1.0
2020-08-16 20:36:00,Select,23.21,1.0
2020-08-24 18:30:00,Comfort,20.62,1.0
2020-08-08 05:50:00,Van,27.45,1.0
2020-08-20 05:42:00,UberX,13.53,1.0
2020-08-20 07:22:00,UberX Protection,14.64,1.0
2020-08-22 20:02:00,Black,48.54,1.2
2020-08-10 11:16:00,Select,23.50,1.0


In [46]:
df_uberx = df[df.name=='UberX']
series_15m = df_uberx.resample("30MIN").price.mean()

In [50]:
df_uberx[["price"]].describe().T

,count,mean,std,min,25%,50%,75%,max
price,12452.0,15.63812,3.163977,13.22,14.07,14.7,15.51,45.38


In [85]:
#@title
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(df_uberx.index), y=list(df_uberx.price), line=dict(color='red', width=2), name='Cena przejazdu' ))
fig.add_trace(go.Scatter(x=list(series_15m.index), y=list(series_15m.values), line=dict(color='green', width=2), name='Uśredniona cena 30-minutowa' ))

fig.update_layout(title_text="UBER - Kwota przejazdu", height=800)

fig.update_layout(
    
    shapes=[
        dict(
            fillcolor="rgba(76, 175, 80, 0.1)",
            line={"width": 0},
            type="rect",
            x0="2020-08-23 19:30",
            x1="2020-08-23 23:50",
            xref="x",
            y0=0,
            y1=0.98,
            yref="paper"
        )],

    annotations=[
        dict(
            x="2020-08-23 21:00",
            y=50,
            xref="x",
            yref="y",
            text="Finał LM",
            font=dict(
              size=20,
              color="#F00"
            ),
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        )
    ],

    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1D",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="7D",
                     step="day",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ), 
    template="plotly_white",
)

initial_range = ['2020-08-22 00:00:00','2020-08-24 23:59:59']
fig['layout']['xaxis'].update(range=initial_range)
fig.show()

In [86]:
#@title
fig = go.Figure(data=[go.Histogram(x=df_uberx.price, opacity=0.8, marker=dict(color='red') )])
fig.update_layout(
    title_text='Rozkład cen', 
    xaxis_title_text='Cena przejazdu',
    yaxis_title_text='', 
    template="plotly_white",
    bargap=0.2,
    bargroupgap=0.1
)
fig.show()